Задание 1
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

если источник traffic_source равен Yandex или Google, то в source_type ставится organic;

для источников paid и email из России ставим ad;

для источников paid и email не из России ставим other;

все остальные варианты берём из traffic_source без изменений.

Задание 1 (через функцию)

In [ ]:
import pandas as pd

In [ ]:
def classify_traffic(row):

  if row['traffic_source'] in ['yandex', 'google']:
    return 'organic'

  elif (row['region'] == 'Russia' and (row['traffic_source'] == 'paid' or row['traffic_source'] ==  'email')):
    return 'ad'

  elif (row['region'] != 'Russia' and (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email')):
    return 'other'

  else:
    return row['traffic_source']


In [ ]:
log = pd.read_csv('visit_log.csv', sep=';')
log.tail()

In [ ]:
log['source_type'] = log.apply(classify_traffic, axis=1)

In [ ]:
log.head(30)

______________________________________________________________________

Задание 2
В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис. Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [ ]:
import re

In [ ]:
urls = pd.read_csv('URLs.txt')
urls.head()

In [ ]:
regex = r'[/]\d{8}-'

In [ ]:
url1 = urls[urls.url.str.contains(regex)]
url1.head(20)

In [ ]:
# Тест - проверка, что перед 8 цифрами отфильтровался только символ /

url2 = url1[url1.url.str.contains(r'[^/]\d{8}]')]
url2.head()

----------------------------------

Задание 3
Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.



In [ ]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings.tail()

In [ ]:
# Функция вычисления разницы между максимальным и минимальным значениями
# столбца timestamp

def groupby_time_life(data):
    return data.timestamp.max() - data.timestamp.min()

In [ ]:
# Расчет количества оценок каждого пользователя
df_marks1 = df_ratings.groupby('userId').count()

In [ ]:
# Форматирование. Оставляем  нужные столбцы
df_marks3 = df_marks1.filter(items = ['userId', 'movieId'])

In [ ]:
# Форматирование. Переименовываем столбцы
df_marks4 = df_marks3.rename(columns={'movieId': 'marks'})

In [ ]:
# К исходному датафрейму с рейтингами присоединяем столбец marks
# (значение уникально для userId после группировки)

df1 = df_ratings.merge(df_marks4, how='left', on = 'userId')
df1.head()

In [ ]:
# Оставляем строки только тех userId, у кого оценок больше 100

df2 = df1[ df1.marks >=100.0]

In [ ]:
# Из общего количества 100004 строк с оценками осталось
len(df2)

In [ ]:
# Вычисление времени жизни пользователя через агрегирующую функцию
# groupby_time_life при группировке по userId

df_3 = df2.groupby('userId').apply(groupby_time_life)

# Перевод результата Series в DataFrame
df_4 = df_3. to_frame (name='time_sec')

# Перевод секунд в года для каждого пользователя. Сортировка
df_4['time_year'] = round( df_4['time_sec']/60/60/24/365 , 1)
df_4.sort_values('time_sec', ascending = False)


In [ ]:
# Расчет среднего времени жизни пользователей (по всем строчкам)
medium_time_life = round(df_4['time_year'].mean(), 1)
print(f' Среднее время жизни пользователя:   {medium_time_life} года')

_______________________________________________________________________

Задание 4
Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе «Материалы для лекции “Продвинутый pandas”» ---- Ноутбуки к лекции «Продвинутый pandas»).
Нужно сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;

аналогичную таблицу по типам выручки с указанием адреса клиента.
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

In [ ]:
# Сводная таблица с адресами клиентов (присоединение через left join к справочнику)
df_1 = client_base.merge(rzd, how = 'left', on = 'client_id')
df_2 = df_1.merge(auto, how = 'left', on = 'client_id' )
df_3 = df_2.merge(air, how = 'left', on = 'client_id')
df_3 = df_3.fillna(0)
df_3

In [287]:
# Обьединенная таблица без данных о адресах клиентов через outer join
df_4 = rzd.merge(auto, how = 'outer', on = 'client_id')
df_5 = df_4.merge(air, how = 'outer', on = 'client_id')
df_5 = df_5.fillna(0)
df_5

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0
